# 2017-05-15
---

Progress of my work : After some weeks of reverse-engineering the OpenRetina scripts, I understood them enough to work direclty on them. L. Perrinet and I built some testing scripts that allowed me to try these scripts. The first one, with a fully-noised input, was run successfully. I still have some problems to run the second one, that use a local camera as an input.

- [ ] Run the local testing script
    - [ ] Solve the empty capture problem
    
Since the code() function seems to receive empty inputs, the problem may come from the run() function.  
Trying to find the source of that problem.  

    def run(self):
        (...)
        print('dbg_key1', self.frame)
        data = self.code(self.frame)
        (...)
        
Output : 

    dbg_key1 None
    
Then :

    def grab(self):
        (...)
        print('dbg_key1', frame)
        print('dbg_key2', self.cap.read())
        (...)
        
Output : 
    
    dbg_key1 None
    dbg_key2 (False, None)
    
The problem seem to come from the PhotoReceptor() class, as trying to call it alone don't work : 

    from openRetina import openRetina
    from openRetina import PhotoReceptor
    import cv2

    cam = PhotoReceptor(640,480)
    cam.grab()
    
Output : 

    Unable to capture video
    dbg_key1 (False, None)

I had some internet problems that slowed down my work. My computer get randomly disconnected, forcing me to restart it to make it work again.  
The script problem come from the PhotoReceptor class, as : 

    (...)
    if not self.cap.isOpened(): 
        print('Camera is not opened')
        stop
    print('dbgkey1')
    (...)
    
Output :

    Camera is not opened
    
I'm missing something. While my own *camera.py* and the official *openRetina* PhotoReceptor classes are fairly similar, the openRetina one don't work. 

    Before a downscale of 1 dim1 : 480, dim2 : 640
    Using OpenCV
    After a downscale of 1, dim1 : 0.0, dim2 : 0.0
    
The null dimensions surely come from the closed camera. Why is the camera not opening?  
OpenCV documentation suggested to use the following function to force it, but it doesn't change anything (see : http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html#capture-video-from-camera) :

    self.cap.open(0)
    
When used through Cheese (capturing program), the camera open and record just fine.  
This command line give informations about the devices connected to the computer's USB bus, including the camera (first one to appear, see : https://help.ubuntu.com/community/Webcam) :

    lsusb -v
    
Even when using a simple script that I already used and that was working, the camera don't launch and no error is given :

    import cv2

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print('Cam not opnd')
    while(True):
        ret,frame = cap.read()
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

I'm using the last openCV version (3.2.0.7), maybe that's what is causing that bug. But the VideoCapture is still called the same way than the previous versions I used (API of the function for that version : http://docs.opencv.org/3.0-beta/modules/videoio/doc/reading_and_writing_video.html?highlight=videocapture#videocapture-videocapture).  
    The fact that the camera isn't opening on my personnal computer that use the same openCV version reinforce the idea that it is linked to the openCV version used (as suggested here : http://answers.opencv.org/question/88857/videocapture-is-not-working-in-opencv-300-or-the-310-in-windows-7/ and suggested by lot of recent questions on internet about the same issue, eg.: http://answers.opencv.org/question/136678/opencv-error-assertion-failed-sizewidth0-sizeheight0-in-cvimshow-python/, https://github.com/opencv/opencv/issues/8471 and https://github.com/opencv/opencv/issues/8358).  
I need to ask L. Perrinet which version is installed on his computer, since the scripts worked on it.  
All these links suggest that the issue is caused by the version 3.2.x, so I tried to downgrade to 3.1.x, the previous version :

    pip3 uninstall opencv-python
    pip install opencv-python==3.1.0 --user
    
But it didn't solved the issue.

# Outline
---

- [ ] Run the local testing script
    - [ ] Solve the empy capture problem